In [4]:
#This utility opens the leafra SQLLite DB - dumps the chunks of a document from the DB - which can have overlaps - and merges them. 
#then it compares with the original document. 
#it uses the schema defined in bool SQLiteDatabase::createRAGTables() in leafra_sqlite.cpp 

# All imports
import sqlite3
import os
import datetime
from urllib.parse import urlparse
from difflib import SequenceMatcher, unified_diff
from io import BytesIO

# Optional imports that may not be available
try:
    import pypdfium2 as pdfium
    PDFIUM_AVAILABLE = True
except ImportError:
    PDFIUM_AVAILABLE = False

try:
    import requests
    REQUESTS_AVAILABLE = True
except ImportError:
    REQUESTS_AVAILABLE = False

leafradbpath = "/Users/arifdikici/Library/Application Support/LeafraSDK/leafra.db"

def parse_chunks(cursor, doc_id):
    """
    Parse and return chunks for a specific document from the database.
    
    Args:
        cursor: SQLite cursor object
        doc_id: Document ID to get chunks for
        
    Returns:
        list: List of chunk dictionaries with chunk_no, text, token_size, size, page_number
    """
    # Get all chunks for this document, ordered by chunk number (include page numbers)
    cursor.execute("""
        SELECT chunk_no, chunk_text, chunk_token_size, chunk_size, chunk_page_number 
        FROM chunks 
        WHERE doc_id = ? 
        ORDER BY chunk_no
    """, (doc_id,))
    
    chunks = cursor.fetchall()
    print(f"Number of chunks: {len(chunks)}")
    
    # Store chunks in memory
    doc_chunks = []
    for chunk in chunks:
        chunk_no, chunk_text, token_size, size, page_number = chunk
        doc_chunks.append({
            'chunk_no': chunk_no,
            'text': chunk_text,
            'token_size': token_size,
            'size': size,
            'page_number': page_number
        })
        print(f"Chunk {chunk_no}: {token_size} tokens, {size} bytes, page {page_number}")
    
    return doc_chunks


def save_chunks_to_file(chunks, filename, output_dir="raw_chunks"):
    """
    Save individual chunks to separate text files.
    
    Args:
        chunks: List of chunk dictionaries
        filename: Base filename (with extension) for the document
        output_dir: Directory to save chunks (default: "raw_chunks")
        
    Returns:
        bool: True if successful, False if no chunks to save
    """
    if not chunks:
        print("No chunks found for document")
        return False
    
    # Create directory for chunks if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Extract base filename without extension
    base_filename = os.path.splitext(filename)[0]
    
    # Save each chunk to a separate file
    for chunk in chunks:
        chunk_filename = f"{base_filename}_rawchunk_{chunk['chunk_no']}.txt"
        chunk_path = os.path.join(output_dir, chunk_filename)
        
        # Write chunk text as binary to preserve exact bytes
        with open(chunk_path, 'wb') as f:
            f.write(chunk['text'].encode('utf-8'))
    
    print(f"Saved {len(chunks)} chunks to {output_dir}/")
    return True

# Helper function to find which chunk contains a specific position
def find_chunk_for_position(chunks, position, merged_text):
    """Find which chunk likely contains the text at the given position"""
    # This is an approximation since we've merged the chunks
    current_pos = 0
    
    for chunk in chunks:
        chunk_len = len(chunk['text'])
        if current_pos <= position < current_pos + chunk_len:
            # Get page number from database if available
            return {
                'chunk_no': chunk['chunk_no'],
                'page': chunk.get('page_number', 'unknown'),
                'start_pos': current_pos,
                'end_pos': current_pos + chunk_len
            }
        current_pos += chunk_len + 1  # +1 for space added during merging
    
    return None


def merge_chunks(chunks):
    """
    Merge chunks handling overlaps to reconstruct the original document text.
    
    Args:
        chunks: List of chunk dictionaries with 'text', 'chunk_no', and other metadata
        
    Returns:
        str: Merged text with overlaps properly handled
        
    Note:
        This function handles word-level overlap detection between consecutive chunks
        and preserves original formatting as much as possible while removing duplicates.
    """
    if not chunks:
        return ""
    
    if len(chunks) == 1:
        return chunks[0]['text']
    
    # Start with first chunk
    merged_text = chunks[0]['text']
    
    for i in range(1, len(chunks)):
        current_chunk = chunks[i]['text']
        chunk_no = chunks[i]['chunk_no']
        
        # Split chunks into words for overlap detection only
        prev_words = merged_text.split()
        curr_words = current_chunk.split()
        
        # Find overlap by comparing word sequences from end of previous to start of current
        max_overlap_len = min(len(prev_words), len(curr_words))
        overlap_size = 0
        
        for overlap_len in range(max_overlap_len, 0, -1):
            if prev_words[-overlap_len:] == curr_words[:overlap_len]:
                overlap_size = overlap_len
                break
        
        if overlap_size == 0:
            print(f"WARNING: No word overlap found between chunks {chunk_no-1} and {chunk_no}")
            # Just concatenate with space, preserving original chunk formatting
            merged_text += " " + current_chunk
        else:
            # Check if overlap preserves word boundaries
            overlap_text_prev = " ".join(prev_words[-overlap_size:])
            overlap_text_curr = " ".join(curr_words[:overlap_size])
            
            if overlap_text_prev != overlap_text_curr:
                print(f"WARNING: Overlap text mismatch between chunks {chunk_no-1} and {chunk_no}")
                print(f"Previous chunk overlap: '{overlap_text_prev}'")
                print(f"Current chunk overlap: '{overlap_text_curr}'")
            
            # Simple approach: find where to cut the current chunk to preserve formatting
            if overlap_size > 0:
                # Find the position in current_chunk where we should start taking text
                # by locating each overlapping word sequentially
                search_pos = 0
                words_found = 0
                
                for word in curr_words[:overlap_size]:
                    word_pos = current_chunk.find(word, search_pos)
                    if word_pos != -1:
                        # Move search position to after this word
                        search_pos = word_pos + len(word)
                        words_found += 1
                    else:
                        break
                
                if words_found == overlap_size:
                    # Successfully found all overlapping words, take text from this position
                    remaining_text = current_chunk[search_pos:]
                    merged_text += remaining_text
                else:
                    # Fallback: use word-based approach
                    merged_text += " " + " ".join(curr_words[overlap_size:])
            else:
                merged_text += " " + current_chunk
    
    print(f"Original total chunks length: {sum(len(c['text']) for c in chunks)}")
    print(f"Merged text length: {len(merged_text)}")
    
    return merged_text

def parse_original_doc(doc_url):
    """
    Parse the original document from URL or local file path.
    
    Args:
        doc_url: URL or file path to the document
        
    Returns:
        str: Extracted text from the document, or empty string if parsing fails
        
    Note:
        Supports PDF and TXT files from both local paths and web URLs.
        Uses pypdfium2 for PDF parsing and basic file reading for TXT files.
    """
    try:
        if not PDFIUM_AVAILABLE:
            print("Error: pypdfium2 not installed. Install with: pip install pypdfium2")
            raise ImportError("pypdfium2 not available")
        
        # Check file extension to determine document type
        parsed_url = urlparse(doc_url)
        print("Parsing document...")

        # Get file extension
        if parsed_url.scheme in ('http', 'https'):
            # Extract extension from URL path
            file_ext = os.path.splitext(parsed_url.path)[1].lower()
        else:
            # Extract extension from local file path
            file_ext = os.path.splitext(doc_url)[1].lower()
        
        original_text = ""
        
        if file_ext == '.pdf':
            # Handle PDF files with pdfium
            if parsed_url.scheme in ('http', 'https'):
                # It's a web URL - download it
                if not REQUESTS_AVAILABLE:
                    print("Error: requests not installed. Install with: pip install requests")
                    raise ImportError("requests not available")
                response = requests.get(doc_url)
                pdf_data = response.content
                pdf_doc = pdfium.PdfDocument(pdf_data)
            else:
                # It's a local file path
                if os.path.exists(doc_url):
                    pdf_doc = pdfium.PdfDocument(doc_url)
                else:
                    print(f"Error: Local file not found: {doc_url}")
                    pdf_doc = None
            
            if pdf_doc is not None:
                # Extract text from all pages using pdfium
                for page_num in range(len(pdf_doc)):
                    page = pdf_doc.get_page(page_num)
                    textpage = page.get_textpage()
                    page_text = textpage.get_text_range()
                    original_text += page_text + "\n"
                    
                    # Clean up page objects
                    textpage.close()
                    page.close()
                
                # Clean up document
                pdf_doc.close()
            else:
                print(f"Error: Could not open PDF document: {doc_url}")
                
        elif file_ext == '.txt':
            # Handle TXT files with simple file reading
            if parsed_url.scheme in ('http', 'https'):
                # It's a web URL - download it
                if not REQUESTS_AVAILABLE:
                    print("Error: requests not installed. Install with: pip install requests")
                    raise ImportError("requests not available")
                response = requests.get(doc_url)
                original_text = response.text
            else:
                # It's a local file path
                if os.path.exists(doc_url):
                    with open(doc_url, 'r', encoding='utf-8') as f:
                        original_text = f.read()
                else:
                    print(f"Error: Local file not found: {doc_url}")
        else:
            print(f"Error: Unsupported file type '{file_ext}' for document: {doc_url}")
            print("Supported types: .pdf, .txt")
        
        return original_text
        
    except Exception as e:
        print(f"Error parsing document {doc_url}: {str(e)}")
        return ""

def difference_analysis(original_text, merged_text, chunks, filename, doc_id, base_filename):
    """
    Perform detailed difference analysis between original and merged texts.
    
    Args:
        original_text: Original document text
        merged_text: Merged chunks text
        chunks: List of chunk dictionaries for mapping differences back to chunks
        filename: Document filename for reporting
        doc_id: Document ID for reporting
        base_filename: Base filename for output files
        
    Returns:
        dict: Analysis results containing similarity ratio, differences count, and report filename
        
    Note:
        Performs word-level and character-level difference detection, maps differences
        back to chunks, and saves detailed analysis report to file.
    """
    # Calculate similarity percentage
    similarity = SequenceMatcher(None, original_text, merged_text).ratio() * 100
    print(f"Text similarity: {similarity:.2f}%")
    
    # Find major differences
    if len(original_text) != len(merged_text):
        print(f"Length difference: Original={len(original_text)}, Merged={len(merged_text)}")
    
    # Show all differences with detailed analysis
    
    # Split into words for better difference detection
    original_words = original_text.split()
    merged_words = merged_text.split()
    
    # Create output list to capture all analysis output
    diff_output = []
    
    # Helper function to add to both console and file output
    def log_output(message):
        print(message)
        diff_output.append(message)
    
    log_output(f"\nDetailed difference analysis:")
    log_output(f"Original word count: {len(original_words)}")
    log_output(f"Merged word count: {len(merged_words)}")
    
    # Find word-level differences
    matcher = SequenceMatcher(None, original_words, merged_words)
    differences_found = 0
    
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag != 'equal':
            differences_found += 1
            log_output(f"\nDifference #{differences_found} - {tag.upper()}:")
            
            if tag == 'delete':
                log_output(f"  Missing in merged (words {i1}-{i2}): {' '.join(original_words[i1:i2])}")
            elif tag == 'insert':
                log_output(f"  Extra in merged (words {j1}-{j2}): {' '.join(merged_words[j1:j2])}")
            elif tag == 'replace':
                log_output(f"  Original (words {i1}-{i2}): {' '.join(original_words[i1:i2])}")
                log_output(f"  Merged (words {j1}-{j2}): {' '.join(merged_words[j1:j2])}")
            
            # Try to map differences back to chunks
            if tag in ['delete', 'replace'] and i1 < len(original_words):
                # Find which chunk(s) contain this difference
                word_position = len(' '.join(original_words[:i1]))
                chunk_info = find_chunk_for_position(chunks, word_position, merged_text)
                if chunk_info:
                    log_output(f"  -> Likely in chunk {chunk_info['chunk_no']} (page {chunk_info.get('page', 'unknown')})")
    
    if differences_found == 0:
        log_output("No word-level differences found (this shouldn't happen if texts don't match)")
    else:
        log_output(f"\nTotal differences found: {differences_found}")
    
    # Show character-level first difference for debugging
    log_output(f"\nFirst character difference:")
    i = 0
    while i < min(len(original_text), len(merged_text)):
        if original_text[i] != merged_text[i]:
            context = 100  # More context for debugging
            start = max(0, i - context)
            end = min(len(original_text), i + context)
            log_output(f"Position {i}:")
            log_output(f"Original: ...{original_text[start:end]}...")
            log_output(f"Merged:   ...{merged_text[start:end]}...")
            break
        i += 1
    
    # Save all difference analysis to file
    diff_filename = f"{base_filename}_differences_merged_vs_original.txt"
    with open(diff_filename, 'w', encoding='utf-8') as f:
        # Add header information
        f.write(f"Difference Analysis Report\n")
        f.write(f"Document: {filename}\n")
        f.write(f"Document ID: {doc_id}\n")
        f.write(f"Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("=" * 50 + "\n\n")
        
        # Add similarity information
        f.write(f"Text similarity: {similarity:.2f}%\n")
        f.write(f"Length difference: Original={len(original_text)}, Merged={len(merged_text)}\n")
        
        # Add all captured output
        for line in diff_output:
            f.write(line + "\n")
    
    print(f"\nDifference analysis saved to: {diff_filename}")
    
    return {
        'similarity': similarity,
        'differences_found': differences_found,
        'report_filename': diff_filename,
        'original_length': len(original_text),
        'merged_length': len(merged_text)
    }




#Main function
def main():
    """
    Main function that processes all documents in the database.
    
    Opens the SQLite database, retrieves all documents, processes their chunks,
    merges them, compares with original documents, and performs difference analysis.
    """
    # Open the SQLite database
    try:
        conn = sqlite3.connect(leafradbpath)
        cursor = conn.cursor()
        print(f"Successfully opened database at {leafradbpath}")
    except sqlite3.Error as e:
        print(f"Error opening database: {e}")
        raise

    #check the schema defined in bool SQLiteDatabase::createRAGTables() in leafra_sqlite.cpp  
    #open the doc table 
    #go through docs one by one 
    #read the chunks of the document using the chunks table into memory 
    # Get schema info
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='docs'")
    docs_schema = cursor.fetchone()
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='chunks'")
    chunks_schema = cursor.fetchone()

    print("\nDocs table schema:")
    print(docs_schema[0])
    print("\nChunks table schema:")
    print(chunks_schema[0])

    # Query all documents
    cursor.execute("SELECT id, filename, url, creation_date, size FROM docs")
    documents = cursor.fetchall()

    print(f"\nFound {len(documents)} documents")

    #for each document - merge the chunks, note that chunks can have overlaps - need figure out the overlaps and merge them without duplicating text. 
    for doc in documents:
        doc_id, filename, url, creation_date, size = doc
        print(f"\nDocument {doc_id}:")
        print(f"Filename: {filename}")
        print(f"URL: {url}")
        print(f"Created: {creation_date}")
        print(f"Size: {size}")
        
        print(f"\nProcessing document {doc_id}: {filename}")
        
        # Parse chunks for this specific document (FIXED: was incorrectly reusing chunks from last document)
        chunks = parse_chunks(cursor, doc_id)
        
        # Save raw chunks to separate files
        if not save_chunks_to_file(chunks, filename):
            continue
        
        # Extract base filename for later use
        base_filename = os.path.splitext(filename)[0]

        # Merge chunks handling overlaps using the merge_chunks function
        merged_text = merge_chunks(chunks)
        
        # Store merged text for later use
        doc_merged_texts = {
            'doc_id': doc_id,
            'merged_text': merged_text
        }

        #now for all documents open the original document - and compare the merged text with the original document. original document's URL is stored in the docs table. 
        #use a helper library for pdfs to parse the original document. Indicate any differences... 
        # Get original document URL from docs table
        query = f"SELECT url FROM docs WHERE id = {doc_id}"
        cursor.execute(query)
        doc_url = cursor.fetchone()[0]
        print(f"Original document URL: {doc_url}")
        if not doc_url:
            print(f"Warning: No URL found for document {doc_id}")
            # Skip processing this document since no URL is available
        else:
            # Parse original document using the parse_original_doc function
            original_text = parse_original_doc(doc_url)

            #save the original text and the merged text to a file
            with open(f"{base_filename}_original.txt", 'w') as f:
                f.write(original_text)
            with open(f"{base_filename}_merged.txt", 'w') as f:
                f.write(merged_text)
            print(f"Saved original text to {base_filename}_original.txt")
            print(f"Saved merged text to {base_filename}_merged.txt")

            # Compare texts
            if original_text == merged_text:
                print(f"Document {doc_id}: Merged chunks match original document exactly!")
            else:
                print(f"Document {doc_id}: Differences found between merged chunks and original")
                
                # Perform detailed difference analysis using the difference_analysis function
                analysis_results = difference_analysis(
                    original_text, merged_text, chunks, 
                    filename, doc_id, base_filename
                )
    
    # Close database connection
    conn.close()
    print("\nDatabase connection closed.")
    print("Processing complete!")



#Calling main 
if __name__ == "__main__":
    main()










Successfully opened database at /Users/arifdikici/Library/Application Support/LeafraSDK/leafra.db

Docs table schema:
CREATE TABLE docs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            filename TEXT NOT NULL,
            url TEXT,
            creation_date DATETIME DEFAULT CURRENT_TIMESTAMP,
            size INTEGER NOT NULL
        )

Chunks table schema:
CREATE TABLE chunks (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            doc_id INTEGER NOT NULL,
            chunk_page_number INTEGER NOT NULL,
            chunk_faiss_id INTEGER,
            chunk_no INTEGER NOT NULL,
            chunk_token_size INTEGER NOT NULL,
            chunk_size INTEGER NOT NULL,
            chunk_text TEXT NOT NULL,
            chunk_embedding BLOB,
            FOREIGN KEY (doc_id) REFERENCES docs(id) ON DELETE CASCADE
        )

Found 1 documents

Document 1:
Filename: cc_introduction_programming.pdf
URL: /Users/arifdikici/Documents/Squirrel/LeafraSDK/example/example_files/cc_in